In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Set up column headers
# procedure_headers are columns with procedure codes
# diagnosis_headers are columns with diagnosis codes

pd.set_option('display.max_columns', None)

procedure_headers = ['PRINPROC','OTHPROC1','OTHPROC2','OTHPROC3','OTHPROC4','OTHPROC5','OTHPROC6','OTHPROC7','OTHPROC8','OTHPROC9','OTHPROC10','OTHPROC11','OTHPROC12','OTHPROC13','OTHPROC14','OTHPROC15','OTHPROC16','OTHPROC17','OTHPROC18','OTHPROC19','OTHPROC20','OTHPROC21','OTHPROC22','OTHPROC23','OTHPROC24','OTHPROC25','OTHPROC26','OTHPROC27','OTHPROC28','OTHPROC29','OTHPROC30']
diagnosis_headers = ['PRINDIAG','OTHDIAG1','OTHDIAG2','OTHDIAG3','OTHDIAG4','OTHDIAG5','OTHDIAG6','OTHDIAG7','OTHDIAG8','OTHDIAG9','OTHDIAG10','OTHDIAG11','OTHDIAG12','OTHDIAG13','OTHDIAG14','OTHDIAG15','OTHDIAG16','OTHDIAG17','OTHDIAG18','OTHDIAG19','OTHDIAG20','OTHDIAG21','OTHDIAG22','OTHDIAG23','OTHDIAG24','OTHDIAG25','OTHDIAG26','OTHDIAG27','OTHDIAG28','OTHDIAG29','OTHDIAG30']
str_headers = procedure_headers + diagnosis_headers + ['MSDRG']

dtype = {str_header: np.str for str_header in str_headers}

In [3]:
# Import data from 2010 to 2017
# Note: this data is already filtered to the 11 hospitals 
# that had heart surgery programs since 2008

discharges = pd.read_csv('KHAll.txt', encoding="ISO-8859-1", dtype=dtype, low_memory=False)

print(len(discharges))

2724538


In [4]:
# Import data from 2008, 2009, and 2018. 
# Those years, some columns differ

discharges08 = pd.read_csv('INP08KHAll.txt', encoding="us-ascii", dtype=dtype, low_memory=False)
discharges09 = pd.read_csv('INP09KHAll.txt', encoding="us-ascii", dtype=dtype, low_memory=False)
discharges18 = pd.read_csv('INP18Q1.txt', encoding="ISO-8859-1", dtype=dtype, low_memory=False)

In [5]:
# Filter out programs if they were not operating for the full year based on AHCA CON forms

def filter_notoperating(r):
    if r['FACLNBR'] == 100007 and r['YEAR'] < 2013:
        return False
    elif r['FACLNBR'] == 100010 and (r['YEAR'] < 2012 or r['YEAR'] > 2014):
        return False
    elif r['FACLNBR'] == 23960096 and r['YEAR'] < 2017:
        return False
    return True

heart_discharges1017 = discharges[discharges.apply(filter_notoperating, axis=1)]
heart_discharges08 = discharges08[discharges08.apply(filter_notoperating, axis=1)]
heart_discharges09 = discharges09[discharges09.apply(filter_notoperating, axis=1)]
heart_discharges18 = discharges18[discharges18.apply(filter_notoperating, axis=1)]

In [6]:
# Filter to patients below age 18 as per AHRQ PDI guidelines
# AGE = 777, 888, or 999 indicates baby below 1 year old for 2018 data

def child_filter(r):
    return (r['AGE'] < 18) or (r['AGE'] == 777) or (r['AGE'] == 888) or (r['AGE'] == 999)

child_discharges1017 = heart_discharges1017[heart_discharges1017.apply(child_filter, axis=1)]
child_discharges08 = heart_discharges08[heart_discharges08.apply(child_filter, axis=1)]
child_discharges09 = heart_discharges09[heart_discharges09.apply(child_filter, axis=1)]
child_discharges18 = heart_discharges18[heart_discharges18.apply(child_filter, axis=1)]

In [7]:
# Merge years of data

child_discharges = pd.concat([child_discharges1017, child_discharges08, child_discharges09, child_discharges18], sort=True)

print(len(child_discharges))

1085031


In [8]:
# Clear memory

child_discharges1017 = child_discharges08 = child_discharges09 = child_discharges18 = None
heart_discharges1017 = heart_discharges08 = heart_discharges09 = heart_discharges18 = None 

In [9]:
# Add ICD coding as column

def get_icd_coding(row):
    if ((row['YEAR'] < 2015) or ((row['YEAR'] == 2015) and row['QTR'] < 4)):
        return 'ICD9'
    else:
        return 'ICD10'

child_discharges['ICD_CODING'] = child_discharges.apply(get_icd_coding, axis=1)

In [10]:
# Import the diagnosis codes for congenital heart disease 
# according to AHRQ specification
ahrq_chd_codes = {
    'ICD9': open('ahrq_codes/icd9_chd_diagnosis.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/icd10_chd_diagnosis.txt', 'r').read().splitlines()
}

# Import the procedure codes for heart surgeries
# according to AHRQ specification
ahrq_surgery_codes = {
    'ICD9': open('ahrq_codes/icd9_chd_procedures.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/icd10_chd_procedures.txt', 'r').read().splitlines()
}

# Import "nonspecific" procedure codes for heart procedures 
# according to AHRQ
# (These are counted only when there is a CHD diagnosis)
ahrq_nonspecific_codes = {
    'ICD9': open('ahrq_codes/icd9_nonspecific_procedures.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/icd10_nonspecific_procedures.txt', 'r').read().splitlines()
}

In [11]:
# Filter to every procedure counted by AHRQ specifications

# Helper function to get the day a procedure occured
import string
d = dict(enumerate(string.ascii_uppercase, 1))

def get_days_from_proc(surgery, procedure_header):
    if procedure_header == 'PRINPROC':
        return surgery['DAYSPROC']
    else:
        proc_num = int(procedure_header.replace('OTHPROC', ''))
        if proc_num > 9:
            # After procedure number 10, column is coded as letters
            # 10 is A, 11 is B, 12 is C...
            return surgery['DAYS_PRO_' + d[proc_num-9]]
        else:
            return surgery['DAYS_PROC' + str(proc_num)]


# Helper function to keep track of the initial heart procedures for each discharge
initprocs = {}
def store_proc(ph, row):
    sys_recid = row['SYS_RECID']
    if initprocs.get(sys_recid) is None:
        initprocs[sys_recid] = ph
    elif get_days_from_proc(row, initprocs.get(sys_recid)) > get_days_from_proc(row, ph):
        initprocs[sys_recid] = ph

# Helper function to check if a patient was diagnosed with a 
# congenital heart condition according to AHRQ specs
def has_chd(row):
    icd_coding = row['ICD_CODING']
    for diagnosis_header in diagnosis_headers:
        if any(chd_code == str(row[diagnosis_header]) for chd_code in ahrq_chd_codes[icd_coding]):
            return True
    return False   

# Main filter function
def has_hsurgery(row):
    icd_coding = row['ICD_CODING']
    
    # For every surgery, check each column that can have a procedure
    for procedure_header in procedure_headers:
        # Check if the column has data
        if isinstance(row[procedure_header], str):
            # Check if the procedure code in that column is a AHRQ heart surgery
            if row[procedure_header] in ahrq_surgery_codes[icd_coding]:
                # If so, keep track of that procedure
                store_proc(procedure_header, row)
            # Otherwise, check if the procedure code is a "nonspecific" procedure
            # AND if the surgery has any listed diagnoses for congenital heart disease
            elif row[procedure_header] in ahrq_nonspecific_codes[icd_coding] and has_chd(row):
                # If so, keep track of that procedure
                store_proc(procedure_header, row)

    # If we found any AHRQ surgeries, this discharge counts
    if initprocs.get(row['SYS_RECID']) is not None:
        return True
    return False

ahrq_inclusions = child_discharges[child_discharges.apply(has_hsurgery, axis=1)]
len(ahrq_inclusions)

13427

In [12]:
# Identify additional ICD 10 open heart procedures that match
# a 0 2 _ _ 0 _ _ pattern and have a CHD diagnosis

p = re.compile('02..0..')

def has_open_heart(row):
    for procedure_header in procedure_headers:
        if (isinstance(row[procedure_header], str) and p.match(row[procedure_header])):
            store_proc(procedure_header, row)
            return has_chd(row)
    return False

icd10_child_discharges = child_discharges[child_discharges['ICD_CODING'] == 'ICD10']
icd10_openh_surgeries = icd10_child_discharges[icd10_child_discharges.apply(has_open_heart, axis=1)]

print(len(icd10_openh_surgeries))

2948


In [13]:
# Merge data before applying exclusions

merge_surgeries = pd.concat([ahrq_inclusions, icd10_openh_surgeries]).drop_duplicates(['SYS_RECID'])
print(len(merge_surgeries))

13670


In [14]:
# A list of recent surgeon codes based on STS hospital pages
# STS lists surgeons operating from 2017-2014, which covers 
# ICD 10 years 17, 16, 15. These codes are only used to filter ICD 10 data.

hospital_surgeons = {
    # Florida Hospital
    100007: ['ME54789', 'ME112169', 'ME133085'], 
    # St. Mary's (Not operating)
    100010: [],
    # Jackson Memorial
    100022: ['ME82017'],
    # Joe DiMaggio
    100038: ['ME114052', 'ME31256', 'ME97233', 'ME130689'],
    # St. Joseph's
    100075: ['ME128598', 'ME122521', 'ME118772', 'ME121592', 'ME118912'],
    # Wolfson Children's
    100088: ['ME109640', 'ME50651', 'ME95724', 'ME118772', 'ME137039'], 
    # UF Health
    100113: ['ME72279', 'ME95724', 'MFC1730', 'ME136295', 'ME109640', 'ME120687'],
    # All Children's
    100250: ['ME64472', 'ME121358', 'ME132548', 'ME42224', 'ME112169', 'ME59091', 'ME91269'],
    # Nicklaus Children's
    110199: ['ME68271', 'ME129510', 'ME76954'],
    # Arnold Palmer
    120001: ['ME91478', 'ME68295'],
    # Nemours
    23960096: ['ME130993', 'ME118912']
}

In [15]:
# Filter ICD 10 discharges if they do not list a heart surgeon
# as the attending physician or one of the operating doctors

def has_surgeon(row):
    if row['ICD_CODING'] == 'ICD9':
        return True
    else:
        if row['FACLNBR'] in hospital_surgeons:
            this_surgeons = hospital_surgeons[row['FACLNBR']]
            if row['ATTEN_PHYI'] in this_surgeons or row['OPER_PHYID'] in this_surgeons or row['OTHOPER_PH'] in this_surgeons:
                return True
        return False

merge_w_surgeon = merge_surgeries[merge_surgeries.apply(has_surgeon, axis=1)]
print(len(merge_w_surgeon))

13236


In [16]:
# Filter out cases that fall under AHRQ's exclusions

# Import/set ICD codes used in exclusions
_3AP = {
    'ICD9': ['35.00', '35.01', '35.02', '35.03', '35.04'],
    'ICD10': ['02CF4ZZ', '02CG4ZZ', '02CH4ZZ', '02CJ4ZZ', '02NF4ZZ', '02NG4ZZ', '02NH4ZZ', '02NJ4ZZ']
}
_3BP = {
    'ICD9': ['35.41', '35.42'],
    'ICD10': ['02B50ZZ', '02B54ZZ']
}
_3CP = {
    'ICD9': ['35.51', '35.71'],
    'ICD10': ['02Q50ZZ', '02Q54ZZ', '02U50JZ']
}
_3DP = {
    'ICD9': ['35.53', '35.72'],
    'ICD10': ['02QM0ZZ', '02QM4ZZ', '02RM0JZ', '02UM0JZ', '02UM4JZ']
}
_3FP = {
    'ICD9': ['38.84', '38.85', '39.59'],
    'ICD10': ['02LR0CT', '02LR0DT', '02LR0ZT', '02LR4CT', '02LR4DT', '02LR4ZT', '02SP0ZZ', '02SQ0ZZ', '02SR0ZZ', '02SS0ZZ', '02ST0ZZ', '02SV0ZZ', '02SW0ZZ', '04L00CZ', '04L00DZ', '04L00ZZ', '04L04CZ', '04L04DZ', '04L04ZZ', '04S00ZZ', '04S04ZZ', '06S00ZZ', '06S04ZZ']
}
_3EP = {
    'ICD9': ['38.85'],
    'ICD10': ['02LR0CT', '02LR0DT', '02LR0ZT', '02LR4CT', '02LR4DT', '02LR4ZT']
}
_3D = {
    'ICD9': ['747.0'],
    'ICD10': ['Q25.0']
}
_4P = {
    'ICD9': ['35.41', '35.52'],
    'ICD10': ['02U54JZ']
}
_5P = {
    'ICD9': ['39.61'],
    'ICD10': ['5A1221Z']
}
_5D = {
    'ICD9': ['745.4', '745.5'],
    'ICD10': ['Q21.0', 'Q21.1']
}
_6P = {
    'ICD9': ['37.21', '37.22', '37.23', '88.42', '88.43', '88.44', '88.50', '88.51', '88.52', '88.53', '88.54', '88.55', '88.56', '88.57', '88.58'],
    'ICD10': open('ahrq_codes/icd10_cath.txt', 'r').read().splitlines()
}
_7P = {
    'ICD9': ['37.5', '37.51', '37.52'],
    'ICD10': ['02YA0Z0', '02YA0Z1']
}
_4D = {
    'ICD9': ['765.00', '765.01', '765.02', '765.03', '765.04', '765.05', '765.06', '765.07', '765.08', '765.09', '765.10', '765.11', '765.12', '765.13', '765.14', '765.15', '765.16', '765.17', '765.18', '765.19'],
    'ICD10': ['P07.00', 'P07.01', 'P07.02', 'P07.03', 'P07.10', 'P07.14', 'P07.15', 'P07.16', 'P07.17', 'P07.18', 'P07.20', 'P07.30']
}
less500_codes = {
    'ICD9': ['764.01', '764.11', '764.21', '764.91', '765.01', '765.11', 'V2131'],
    'ICD10': ['P05.01', 'P05.11', 'P07.01']
}
mdc_14_codes = ['765', '766', '767', '768', '769', '770', '774', '775', '776', '777', '778', '779', '780', '781', '782', '998']
LIVEBND = {
    'ICD9': ['V3000', 'V3401', 'V3001', 'V3500', 'V3100', 'V3501', 'V3101', 'V3600', 'V3200', 'V3601', 'V3201', 'V3700', 'V3300', 'V3701', 'V3301', 'V3900', 'V3400', 'V3901'],
    'ICD10': ['Z3800', 'Z3863', 'Z3801', 'Z3864', 'Z3830', 'Z3865', 'Z3831', 'Z3866', 'Z3861', 'Z3868', 'Z3862', 'Z3869']
}
V29D = {
    'ICD9': ['V290', 'V291', 'V292', 'V293', 'V298', 'V299'],
    'ICD10': [] # Not a criteria for neonate for ICD 10
}

# Helper function to determine if neonate
# A neonate is defined as any discharge with either:
#  • age in days at admission between zero and 28 days (inclusive);
#  • age in days missing and age in years equal to zero and either:
#  • an admission type of newborn (SID ATYPE=4);
#  • with any-listed ICD-CM diagnosis codes for in-hospital live birth;
#  • with any-listed ICD-CM diagnosis codes for neonatal observation and evaluation
def is_neonate(age, diagnoses, atype, icd_coding):
    if age != 0 or age != 777 or age != 888 or age != 999:
        return False
    if (atype == 4) or any(diagnosis in LIVEBND[icd_coding] for diagnosis in diagnoses) or any(diagnosis in V29D[icd_coding] for diagnosis in diagnoses):
        return True

# Helper function to determine if a given procedure is the only heart
# surgery in the list
def no_other_chd(chd_procedures, only):
    return len(chd_procedures) > 0 and all(chd_procedure in only for chd_procedure in chd_procedures)

# Helper function to determine if catheterization (6P) 
# without anylisted ICD-CM procedure for extracorporeal circulation (5P)
def cath_without_ecc(procedures, icd_coding):
    if any(procedure in _6P[icd_coding] for procedure in procedures): 
        return all(procedure not in _5P[icd_coding] for procedure in procedures)
    else:
        return False

# Helper function to see if diagnosis with PDA (3D) with only other 
# heart diagnoses are PDA (3D) and VSD (5D) with any procedure code
# for occlusion of a thoracic vessel (3EP) as the only procedure
def pda_star(chd_diagnoses, chd_procedures, icd_coding):
    return len(chd_diagnoses) > 0 \
        and any(pda_code in chd_diagnoses for pda_code in _3D[icd_coding]) \
        and all(diagnosis in (_3D[icd_coding] + _5D[icd_coding]) for diagnosis in chd_diagnoses) \
        and no_other_chd(chd_procedures, _3EP[icd_coding])
    
# Main function to apply exclusions
def ahrq_exclusions(row):
    icd_coding = row['ICD_CODING']
    diagnoses = [row[diagnosis_header] for diagnosis_header in diagnosis_headers]
    procedures = [row[procedure_header] for procedure_header in procedure_headers]
    chd_diagnoses = list(set(diagnoses).intersection(set(ahrq_chd_codes[icd_coding])))
    chd_procedures = list(set(procedures).intersection(set(ahrq_surgery_codes[icd_coding] + ahrq_nonspecific_codes[icd_coding])))
    
    '''
    with any-listed ICD-CM procedure codes for closed heart valvotomy (3AP) as the only congenital heart disease 
    procedure and any-listed ICD-CM procedure codes for catheterization (6P) without anylisted ICD-CM procedure 
    codes for extracorporeal circulation (5P)
    '''
    if no_other_chd(chd_procedures, _3AP[icd_coding]) and cath_without_ecc(procedures, icd_coding):
        return False

    '''
    with any-listed ICD-CM procedure codes for atrial septal enlargement (3BP) as the only congenital
    heart disease procedure and any-listed ICD-CM procedure codes for catheterization (6P) without anylisted
    ICD-CM procedure codes for extracorporeal circulation (5P)
    '''
    if no_other_chd(chd_procedures, _3BP[icd_coding]) and cath_without_ecc(procedures, icd_coding):
        return False
    
    '''
    with any-listed ICD-CM procedure codes for atrial septal defect repair (3CP) as the only congenital
    heart disease procedure and any-listed ICD-CM procedure codes for catheterization (6P) without anylisted
    ICD-CM procedure codes for extracorporeal circulation (5P)
    '''
    if no_other_chd(chd_procedures, _3CP[icd_coding]) and cath_without_ecc(procedures, icd_coding):
        return False
    
    '''
    with any-listed ICD-CM procedure codes for ventricular septal defect repair (3DP) as the only
    congenital heart disease procedure and any-listed ICD-CM procedure codes for catheterization (6P)
    without any-listed ICD-CM procedure codes for extracorporeal circulation (5P)
    '''
    if no_other_chd(chd_procedures, _3DP[icd_coding]) and cath_without_ecc(procedures, icd_coding):
        return False
    
    '''
    with any-listed ICD-CM procedure codes for other surgical occlusion (3FP) as the only congenital
    heart disease procedure and any-listed ICD-CM procedure codes for catheterization (6P) without anylisted
    ICD-CM procedure codes for extracorporeal circulation (5P)
    '''
    if no_other_chd(chd_procedures, _3FP[icd_coding]) and cath_without_ecc(procedures, icd_coding):
        return False
    
    '''
    with patent ductus arteriosus (PDA) (3D) and any-listed ICD-CM procedure codes for catheterization
    (6P) without any-listed ICD-CM procedure codes for extracorporeal circulation (5P)
    '''
    if any(procedure in _3D[icd_coding] for procedure in procedures) and cath_without_ecc(procedures, icd_coding):
        return False
    
    '''
    with any-listed ICD-CM procedure codes for atrial septal defect repair and enlargement (4P) as the
    only congenital heart disease procedure without any-listed ICD-CM procedure codes for extracorporeal
    circulation (5P)
    '''
    if no_other_chd(chd_procedures, _4P[icd_coding]) and not any(procedure in _5P[icd_coding] for procedure in procedures):
        return False
    
    '''
    with any-listed ICD-CM diagnosis codes for premature infant (4D) with PDA†
    '''
    if any(diagnosis in _4D[icd_coding] for diagnosis in diagnoses) and pda_star(chd_diagnoses, chd_procedures, icd_coding):
        return False
    
    '''
    with any-listed ICD-CM diagnosis codes for atrial septal defect or ventricular septal defect (5D) and PDA†
    '''
    if any(diagnosis in _5D[icd_coding] for diagnosis in diagnoses) and pda_star(chd_diagnoses, chd_procedures, icd_coding):
        return False
    
    '''
    age less than or equal to 30 days with PDA†
    NOTE: age data is not presented in days, on consultation with experts, we decided to filter age less than 1 year
    '''
    if pda_star(chd_diagnoses, chd_procedures, icd_coding) and row['AGE'] < 1:
        return False

    '''
    neonates with birth weight less than 500 grams (Birth Weight Category 1)
    '''
    if is_neonate(row['AGE'], diagnoses, row['ADM_PRIOR'], icd_coding) and any(diagnosis in less500_codes[icd_coding] for diagnosis in diagnoses):
        return False
    
    '''
    MDC 14 (pregnancy, childbirth and pueperium)
    '''
    if row['MSDRG'] in mdc_14_codes:
        return False
    
    '''
    transferring to another short-term hospital (DISP=2)
    '''
    if row['DISCHSTAT'] == 2:
        return False

    '''
    with missing discharge disposition (DISP=missing), gender (SEX=missing), age (AGE=missing),
    quarter (DQTR=missing), year (YEAR=missing) or principal diagnosis (DX1=missing)
    '''
    if pd.isnull(row['DISCHSTAT']) or pd.isnull(row['AGE']) or pd.isnull(row['QTR']) or pd.isnull(row['YEAR']) or pd.isnull(row['PRINDIAG']):
        return False
    
    if row['YEAR']:
        if row['YEAR'] > 2009 and not row['SEX']:
            return False
        if row['YEAR'] < 2010 and not row['GENDER']:
            return False
    else: 
        return False

    return True
    

final_surgeries = merge_w_surgeon[merge_w_surgeon.apply(ahrq_exclusions, axis=1)]
print(len(final_surgeries))

11228


In [17]:
# Add death as a column

def has_deceased(row):
    return row['DISCHSTAT'] == 20

final_surgeries = final_surgeries.assign(HAS_DECEASED=final_surgeries.apply(has_deceased, axis=1).values)

print(len(final_surgeries[final_surgeries['HAS_DECEASED'] == True]))

print(len(final_surgeries[final_surgeries['HAS_DECEASED'] == True]) / len(final_surgeries))

341
0.030370502315639474


In [18]:
# Add length of stay after the first heart procedure as a column

def get_los(surgery, procs):
    return surgery['LOSDAYS'] - get_days_from_proc(surgery, procs[surgery['SYS_RECID']])

final_surgeries = final_surgeries.assign(HLOS=final_surgeries.apply(lambda r: get_los(r, initprocs), axis=1).values)

print(final_surgeries['HLOS'].median())

7.0


In [19]:
# Add ECMO after heart surgery as a column

# Note: ICD 10 ecmo code changes in fourth quarter of 2018, 
# but does not apply to our data
ecmo_coding = {
    'ICD9': ['39.65'],
    'ICD10': ['5A15223']
}

def has_ecmo(row):
    icd_coding = row['ICD_CODING']
    for procedure_header in procedure_headers:
        if row[procedure_header] in ecmo_coding[icd_coding]:
            if get_days_from_proc(row, procedure_header) >= (row['LOSDAYS'] - row['HLOS']):
                return True
    return False

final_surgeries = final_surgeries.assign(HAS_ECMO=final_surgeries.apply(has_ecmo, axis=1).values)
    
print(len(final_surgeries[final_surgeries['HAS_ECMO'] == True]))

261


In [20]:
# Wound disruption
disruption_coding = {
    'ICD9': '998.3',
    'ICD10': 'T81.3'
}

def has_disruption(row):
    icd_coding = row['ICD_CODING']
    for diagnosis_header in diagnosis_headers:
        if str(row[diagnosis_header]).startswith(disruption_coding[icd_coding]):
            return True
    return False

final_surgeries = final_surgeries.assign(HAS_DISRUPTION=final_surgeries.apply(has_disruption, axis=1))

print(len(final_surgeries[final_surgeries['HAS_DISRUPTION'] == True]))

196


In [21]:
# Add column for AHRQ indicator 10, postoperative sepsis
# using official methodology
# 1 is Yes, 2 is No, 3 is NA (exclude from denominator)

# Import codes from AHRQ spec
SEPTI2D = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/SEPTI2D.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/SEPTI2D.txt', 'r').read().splitlines()
}
INFECID = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/INFECID.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/INFECID.txt', 'r').read().splitlines()
}
DRG1C = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/DRG1C.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/DRG1C.txt', 'r').read().splitlines() 
}
DRG2C = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/DRG2C.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/DRG2C.txt', 'r').read().splitlines() 
}
DRG3C = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/DRG3C.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/DRG3C.txt', 'r').read().splitlines() 
}
DRG4C = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/DRG4C.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/DRG4C.txt', 'r').read().splitlines() 
}
DRG9C = {
    'ICD9': open('ahrq_codes/PDI10/ICD9/DRG9C.txt', 'r').read().splitlines(),
    'ICD10': open('ahrq_codes/PDI10/ICD10/DRG9C.txt', 'r').read().splitlines() 
}


def pdi10(row):
    icd_coding = row['ICD_CODING']
    diagnoses = [row[diagnosis_header] for diagnosis_header in diagnosis_headers]
    procedures = [row[procedure_header] for procedure_header in procedure_headers]

    # Exclusions
    # • with a principal ICD-10-CM diagnosis code (or secondary diagnosis present on admission) for sepsis
    # (SEPTI2D *), among patients otherwise qualifying for numerator 
    if row['PRINDIAG'] in SEPTI2D[icd_coding]:
        return 3
    
    if row['OTHDIAG1'] in SEPTI2D[icd_coding] and row['POA1'] == 'Y':
        return 3
    
    # with a principal ICD-10-CM diagnosis code for infection (Appendix H: INFECID)
    if row['PRINDIAG'] in INFECID[icd_coding]:
        return 3
    
    # with MS-DRG code for surgical class 4 ( DRG4C *)
    if row['MSDRG'] in DRG4C[icd_coding]:
        return 3
    
    if is_neonate(row['AGE'], diagnoses, row['ADM_PRIOR'], icd_coding):
        return 3
    
    if any(diagnosis in SEPTI2D[icd_coding] for diagnosis in diagnoses[1:]):
        return 1
    else:
        return 2

final_surgeries = final_surgeries.assign(PDI10=final_surgeries.apply(lambda r: pdi10(r), axis=1).values)

def pdi10_risk(row):
    icd_coding = row['ICD_CODING']
    diagnoses = [row[diagnosis_header] for diagnosis_header in diagnosis_headers]
    procedures = [row[procedure_header] for procedure_header in procedure_headers]
    
    if row['PDI10'] == 3:
        return 'NA'
    
    if row['MSDRG'] in DRG1C[icd_coding] and row['ADM_PRIOR'] == 3:
        return '1'
    
    if row['MSDRG'] in DRG1C[icd_coding] and row['ADM_PRIOR'] != 3:
        return '2'
    
    DRG239C = DRG2C[icd_coding] + DRG3C[icd_coding] + DRG9C[icd_coding]
    
    if row['MSDRG'] in DRG239C and row['ADM_PRIOR'] == 3:
        return '3'
    
    if row['MSDRG'] in DRG239C and row['ADM_PRIOR'] != 3:
        return '4'
    
    return '9'

final_surgeries = final_surgeries.assign(PDI10_RISK=final_surgeries.apply(pdi10_risk, axis=1).values)

print(final_surgeries['PDI10'].value_counts())
print(final_surgeries['PDI10_RISK'].value_counts())

2    10853
1      277
3       98
Name: PDI10, dtype: int64
1     5294
2     4102
9     1267
4      381
NA      98
3       86
Name: PDI10_RISK, dtype: int64
